In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

### Data

The dataset contains thousands of images of buildings, forests, glaciers and seas, 4 classes. Out of which <1000 images are selected to train the model. 

Classes :-

In [2]:
os.listdir('./seg_train/')

['buildings', 'forest', 'glacier', 'sea']

Loading data from local storage

In [3]:
from PIL import Image
pxl_size=(200,200)
path='./seg_train/'

Defining reading functions

In [4]:
def read_data(batch_size,path,pxl_size):
    labels=os.listdir(path)
    le=LabelEncoder()
    le.fit(labels)
    max_pts=1000
    for i in range(0,(max_pts//batch_size)):
        x_data=np.zeros((batch_size,pxl_size[1],pxl_size[0],3))
        length=batch_size//4
        p=0
        y_data=[]
        
        for j in range(0,4):
            rel_path=os.path.join(path,labels[j])
            for k in range(i*length,(i+1)*length):
                img=Image.open(rel_path+'/'+os.listdir(rel_path)[k]).resize(pxl_size)
                x_data[p,:,]=np.array(img)/255.0
                y_data.append(labels[j])
                p+=1
                
        y_data=le.transform(y_data)
        ind=np.random.permutation(batch_size)
        x_data=x_data[ind,]
        y_data=y_data[ind]
        yield (x_data,y_data)

### Working with created model

Loading created CNN model trained upon images of cats and dogs.

In [5]:
model=keras.models.load_model('CNN_model.h5')
model.summary()

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 99, 99, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 33, 33, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0       

Removing unnecessary layers

In [6]:
model.pop()
model.pop()
model.pop()
model.pop()
model.pop()
model.summary()

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 99, 99, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 33, 33, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0       

In [7]:
for layer in model.layers:
    layer.trainable=False
model.trainable_variables

[]

Adding necessary layers

In [8]:
h0=keras.layers.BatchNormalization()
h1=keras.layers.Dense(800)
h2=keras.layers.Dropout(0.2)
h3=keras.layers.Dense(400)
h4=keras.layers.Dropout(0.2)
final_layer=keras.layers.Dense(4,activation='softmax')

Assembling the model

In [9]:
model.add(h0)
model.add(h1)
model.add(h2)
model.add(h3)
model.add(h4)
model.add(final_layer)

In [10]:
model.summary()

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 99, 99, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 33, 33, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0       

In [11]:
model.compile('adam','sparse_categorical_crossentropy',['accuracy'])

### Training

In [12]:
batch_size=40
epochs=4
for e in range(0,epochs):
    gen=read_data(batch_size,path,pxl_size)
    model.fit(gen,batch_size=batch_size,epochs=1)

keras.models.save_model(model,'Tranfer_learning.h5',save_format='h5')

25/25 [==============================] - 9s 380ms/step - loss: 0.2851 - accuracy: 0.8910


### Testing

In [13]:
def accuracy(n):
    gen=read_data(n,path,pxl_size)
    x,y=gen.__next__()
    y_pred=np.argmax(model(x),axis=1)
    error=sum(np.not_equal(y,y_pred))/n
    print(f'Error===>{error}')
    print('Accuracy using transfer learning==>',(1-error)*100)

In [14]:
accuracy(500)

Error===>0.124
Accuracy using transfer learning==> 87.6
